In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
aqsol = pd.read_csv('../AqSolDB/aqsoldb_clean_1115.csv')
print(aqsol.shape)
aqsol.head()

(9982, 10)


,id,dataset,setnumber,name,inchi,inchikey,smiles,solubility,sd,ocurrences
0,1,A,3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.0,1
1,2,A,4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.0,1
2,3,A,5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.0,1
3,4,A,8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.0,1
4,5,A,9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.0,1


In [3]:
# henry = pd.read_csv('../HLCDB/henry_clean.csv')
henry = pd.read_csv('../HLCDB/henry_shrinked.csv')
print(henry.shape)
henry.head()

(4513, 6)


,id,avg_kh,med_kh,std_kh,species_id,occurrences
0,1.0,0.000013,0.000013,4.522670e-07,1.0,12.0
1,2.0,0.000110,0.000110,1.294815e-05,2.0,11.0
2,3.0,0.000003,0.000003,0.000000e+00,3.0,2.0
3,4.0,0.000008,0.000008,8.944272e-08,4.0,5.0
4,5.0,10.351111,0.320000,2.949868e+01,6.0,9.0


In [4]:
reference = pd.read_csv('Alastair_MergedDataset.csv')

In [5]:
# species = pd.read_csv('../HLCDB/species.csv', on_bad_lines='skip') # error  on two lines: Error tokenizing data. C error: Expected 7 fields in line 1765, saw 8
# species = pd.read_csv('../HLCDB/species.csv')
species = pd.read_csv('../HLCDB/species_excelled.csv').drop(columns=['Unnamed: 7','formula','subcat_id','trivial'],axis=1)
# print(species.formula[1380])
# print(species.formula[1380].replace('<sub>','').replace('</sub>','_'))
print(species.shape)
species.head()

(4632, 4)


,id,iupac,casrn,inchikey
0,1,oxygen,7782-44-7,MYMOFIZGZYHOMD-UHFFFAOYSA-N
1,2,ozone,10028-15-6,CBENFWSGALASAD-UHFFFAOYSA-N
2,3,hydrogen atom,12385-13-6,YZCKVEUIGOORGS-UHFFFAOYSA-N
3,4,hydrogen,1333-74-0,UFHFLCQGNIYNRP-UHFFFAOYSA-N
4,5,deuterium,7782-39-0,UFHFLCQGNIYNRP-VVKOMZTBSA-N


In [6]:
inchikey_hlc = species['inchikey']

In [7]:
sum(aqsol['inchikey'].value_counts() > 1) # no reptition in aqsol

0

In [8]:
# henry.species table has unique entries
# so there are 1310 species where both HLC and aqsol data is available?

In [9]:
# henry[['id','avg_kh','species_id','occurrences']].merge(species[['id','inchikey']], how='inner', left_on='species_id', right_on='id'
#     ).merge(aqsol,how='inner',left_on='inchikey',right_on='inchikey')

In [10]:
# henry['inchikey'] = henry['id'].apply(lambda x: species['inchikey'][x])
# henry

In [11]:
# merged = aqsol[['solubility','sd','ocurrences','inchikey']].merge(henry[['avg_kh','med_kh','std_kh','occurrences','inchikey','species_id']], how='inner')
# merged

# something wrong w merging, 65 compounds are lost this way

In [12]:
# now let's get compounds from aqsol that are not in merged -> extend species table with it innit
# extend aqsol table with full species (existing and new)

In [13]:
unmatched_hlc = species.loc[~species['inchikey'].isin(aqsol['inchikey'])].copy(deep=True) # inchikey in hlc but not aqsoldb
unmatched_hlc['in_aqsol'] = 0
unmatched_hlc['in_hlc'] = 1
print(unmatched_hlc.shape)
unmatched_hlc.head()

(2226, 6)


,id,iupac,casrn,inchikey,in_aqsol,in_hlc
1,2,ozone,10028-15-6,CBENFWSGALASAD-UHFFFAOYSA-N,0,1
2,3,hydrogen atom,12385-13-6,YZCKVEUIGOORGS-UHFFFAOYSA-N,0,1
3,4,hydrogen,1333-74-0,UFHFLCQGNIYNRP-UHFFFAOYSA-N,0,1
4,5,deuterium,7782-39-0,UFHFLCQGNIYNRP-VVKOMZTBSA-N,0,1
5,6,hydroxyl radical,3352-57-6,TUJKJAMUKRIRHC-UHFFFAOYSA-N,0,1


In [14]:
matched_hlc = species.loc[species['inchikey'].isin(aqsol['inchikey'])].copy(deep=True) # inchikey in hlc AND in aqsoldb
matched_hlc['in_aqsol'] = 1
matched_hlc['in_hlc'] = 1
print(matched_hlc.shape)
matched_hlc.head()

(2406, 6)


,id,iupac,casrn,inchikey,in_aqsol,in_hlc
0,1,oxygen,7782-44-7,MYMOFIZGZYHOMD-UHFFFAOYSA-N,1,1
7,8,hydrogen peroxide,7722-84-1,MHAJPDPJQMAIIY-UHFFFAOYSA-N,1,1
8,9,nitrogen,7727-37-9,IJGRMHOSHXDMSA-UHFFFAOYSA-N,1,1
9,10,ammonia,7664-41-7,QGZKDVFQNNGYKY-UHFFFAOYSA-N,1,1
11,12,hydrazine,302-01-2,OAKJQQAXSVQMHS-UHFFFAOYSA-N,1,1


In [15]:
id_spec_max = species['id'].max()
id_spec_max
# for the aqsol compounds that will be added, counting should start from here

4632

In [16]:
matched_aqsol = aqsol.loc[aqsol['inchikey'].isin(species['inchikey'])].copy(deep=True) # 
matched_aqsol.shape

(2406, 10)

In [17]:
matched_aqsol['species_id'] = np.nan
# matched_aqsol.head()

In [18]:
ids = []
for c, key in enumerate(matched_aqsol['inchikey']):
    id = int(species[species['inchikey']==key]['id'])
    # print(id)
    ids.append(id)
    # matched_aqsol['species_id'][c] = id

matched_aqsol['species_id'] = ids
matched_aqsol.head()

,id,dataset,setnumber,name,inchi,inchikey,smiles,solubility,sd,ocurrences,species_id
5,6,A,10,vinyltoluene,"InChI=1S/C9H10/c1-3-9-6-4-5-8(2)7-9/h3-7H,1H2,2H3",JZHGRUMIRATHIU-UHFFFAOYSA-N,Cc1cccc(C=C)c1,-3.123150,0.000000,1,418
16,17,A,24,2-(4-chloro-2-methylphenoxy)propanoic acid,InChI=1S/C10H11ClO3/c1-6-5-8(11)3-4-9(6)14-7(2...,WNTGYJSOUMFZEP-UHFFFAOYSA-N,CC(Oc1ccc(Cl)cc1C)C(O)=O,-2.466031,0.060621,4,3058
18,19,A,26,1-chloro-3-(trifluoromethyl)benzene,"InChI=1S/C7H4ClF3/c8-6-3-1-2-5(4-6)7(9,10)11/h...",YTCGOUNVIAWCMG-UHFFFAOYSA-N,FC(F)(F)c1cccc(Cl)c1,-3.411514,0.000000,1,3696
21,22,A,29,1-methyl-4-(propan-2-ylidene)cyclohex-1-ene,"InChI=1S/C10H16/c1-8(2)10-6-4-9(3)5-7-10/h4H,5...",MOYAFQVGZZPNRA-UHFFFAOYSA-N,CC(C)=C1CCC(=CC1)C,-4.287343,0.131231,3,433
27,28,A,36,"(1S,5S)-6,6-dimethyl-2-methylidenebicyclo[3.1....","InChI=1S/C10H16/c1-7-4-5-8-6-9(7)10(8,2)3/h8-9...",WTARULDDTDQWMU-UHFFFAOYSA-N,CC1(C)C2CCC(=C)C1C2,-4.292313,1.247787,3,425


In [19]:
unmatched_aqsol = aqsol.loc[~aqsol['inchikey'].isin(species['inchikey'])].copy(deep=True)
# create new species id for these
unmatched_aqsol['species_id'] = np.nan
unmatched_aqsol['species_id'] = np.arange(id_spec_max+1,unmatched_aqsol.shape[0]+id_spec_max+1)
unmatched_aqsol.head()

,id,dataset,setnumber,name,inchi,inchikey,smiles,solubility,sd,ocurrences,species_id
0,1,A,3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.0,1,4633
1,2,A,4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.0,1,4634
2,3,A,5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.0,1,4635
3,4,A,8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.0,1,4636
4,5,A,9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.0,1,4637


In [20]:
new_aqsol = pd.concat([matched_aqsol,unmatched_aqsol],axis=0).sort_values('id')
new_aqsol#.head()

,id,dataset,setnumber,name,inchi,inchikey,smiles,solubility,sd,ocurrences,species_id
0,1,A,3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.000000,1,4633
1,2,A,4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.000000,1,4634
2,3,A,5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.000000,1,4635
3,4,A,8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.000000,1,4636
4,5,A,9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.000000,1,4637
...,...,...,...,...,...,...,...,...,...,...,...
9977,9978,I,84,tetracaine,InChI=1S/C15H24N2O2/c1-4-5-10-16-14-8-6-13(7-9...,GKCBAIGFKIBETG-UHFFFAOYSA-N,C(c1ccc(cc1)NCCCC)(=O)OCCN(C)C,-3.010000,0.000000,1,12206
9978,9979,I,85,tetracycline,InChI=1S/C22H24N2O8/c1-21(31)8-5-4-6-11(25)12(...,OFVLGDICTFRJMM-WESIUVDSSA-N,OC1=C(C(C2=C(O)[C@@](C(C(C(N)=O)=C(O)[C@H]3N(C...,-2.930000,0.000000,1,12207
9979,9980,I,86,thymol,InChI=1S/C10H14O/c1-7(2)9-5-4-8(3)6-10(9)11/h4...,MGSRCZKZVOBKFT-UHFFFAOYSA-N,c1(cc(ccc1C(C)C)C)O,-2.190000,0.019222,3,825
9980,9981,I,93,verapamil,"InChI=1S/C27H38N2O4/c1-20(2)27(19-28,22-10-12-...",SGTNSNPWRIOYBX-UHFFFAOYSA-N,COc1ccc(CCN(C)CCCC(C#N)(C(C)C)c2ccc(OC)c(OC)c2...,-3.980000,0.000000,1,12208


In [21]:
aqsol.shape[0] == new_aqsol.shape[0]

True

In [22]:
# new species df needs couple of things
    # adding new rows to account for unmatched_aqsol ->  added species
    # smiles scraped for unmatched_hlc 
    # smiles simply looked up and added for 

In [23]:
np.array(matched_aqsol[['species_id','smiles']].sort_values('species_id')['smiles'])

array(['O=O', 'OO', 'N#N', ..., 'CC(=O)O[Hg]c1ccccc1', '[Cl-].COCC[Hg+]',
       'CC[Pb](CC)(CC)CC'], dtype=object)

In [24]:
matched_hlc['smiles'] = np.nan
matched_hlc['smiles'] = np.array(matched_aqsol[['species_id','smiles']].sort_values('species_id')['smiles'])
matched_hlc

# doesn't fill in smiles properly cos the dataframe ids are different

,id,iupac,casrn,inchikey,in_aqsol,in_hlc,smiles
0,1,oxygen,7782-44-7,MYMOFIZGZYHOMD-UHFFFAOYSA-N,1,1,O=O
7,8,hydrogen peroxide,7722-84-1,MHAJPDPJQMAIIY-UHFFFAOYSA-N,1,1,OO
8,9,nitrogen,7727-37-9,IJGRMHOSHXDMSA-UHFFFAOYSA-N,1,1,N#N
9,10,ammonia,7664-41-7,QGZKDVFQNNGYKY-UHFFFAOYSA-N,1,1,N
11,12,hydrazine,302-01-2,OAKJQQAXSVQMHS-UHFFFAOYSA-N,1,1,NN
...,...,...,...,...,...,...,...
4602,4601,tetrabutylstannane,1461-25-2,AFCAKJKUYFLYFK-UHFFFAOYSA-N,1,1,CCCC[Sn](CCCC)(CCCC)CCCC
4603,4602,hexabutyldistannoxane,56-35-9,APQHKWPGGHMYKJ-UHFFFAOYSA-N,1,1,CCCC[Sn](CCCC)(CCCC)O[Sn](CCCC)(CCCC)CCCC
4611,4612,phenyl mercuric ethanoate,62-38-4,XEBWQGVWTUSTLN-UHFFFAOYSA-M,1,1,CC(=O)O[Hg]c1ccccc1
4620,4621,2-methoxyethylmercury chloride,123-88-6,VJTAZCKMHINUKO-UHFFFAOYSA-M,1,1,[Cl-].COCC[Hg+]


In [25]:
matched_aqsol[['species_id','smiles']].sort_values('species_id').reset_index(drop=True)

,species_id,smiles
0,1,O=O
1,8,OO
2,9,N#N
3,10,N
4,12,NN
...,...,...
2401,4601,CCCC[Sn](CCCC)(CCCC)CCCC
2402,4602,CCCC[Sn](CCCC)(CCCC)O[Sn](CCCC)(CCCC)CCCC
2403,4612,CC(=O)O[Hg]c1ccccc1
2404,4621,[Cl-].COCC[Hg+]


In [26]:
new_species = pd.concat([matched_hlc, unmatched_hlc],axis=0).sort_values('id')
new_species

,id,iupac,casrn,inchikey,in_aqsol,in_hlc,smiles
0,1,oxygen,7782-44-7,MYMOFIZGZYHOMD-UHFFFAOYSA-N,1,1,O=O
1,2,ozone,10028-15-6,CBENFWSGALASAD-UHFFFAOYSA-N,0,1,NaN
2,3,hydrogen atom,12385-13-6,YZCKVEUIGOORGS-UHFFFAOYSA-N,0,1,NaN
3,4,hydrogen,1333-74-0,UFHFLCQGNIYNRP-UHFFFAOYSA-N,0,1,NaN
4,5,deuterium,7782-39-0,UFHFLCQGNIYNRP-VVKOMZTBSA-N,0,1,NaN
...,...,...,...,...,...,...,...
4627,4628,tetramethyl lead,75-74-1,XOOGZRUBTYCLHG-UHFFFAOYSA-N,0,1,NaN
4628,4629,ethyltrimethylplumbane,1762-26-1,KHQJREYATBQBHY-UHFFFAOYSA-N,0,1,NaN
4629,4630,diethyldimethylplumbane,1762-27-2,OLOAJSHVLXNSQV-UHFFFAOYSA-N,0,1,NaN
4630,4631,triethylmethylplumbane,1762-28-3,KGFRUGHBHNUHOS-UHFFFAOYSA-N,0,1,NaN


In [27]:
# adding smiles to species that were scraped from the internet
# FINISH THIS once scraping is done


In [29]:
# I don't know what's this
# 
# extend it w id (unmatched_aqsol.species_id) and inchikey
# new_species = pd.concat([species, unmatched_aqsol['inchikey']],axis=0).reset_index(drop=True)
# new_species = species.copy(deep=True)
# added_species = pd.DataFrame(columns=species.columns)
# added_species['inchikey'] = unmatched_aqsol['inchikey']
# new_species = pd.concat([species,unmatched_aqsol],axis=0).reset_index(drop=True).drop(['dataset','setnumber','solubility','sd','species_id'],axis=1)

# new_species.ocurrences = new_species.ocurrences.fillna(henry.occurrences)
# new_species.name = new_species.name.fillna(species['iupac'])

# new_species = new_species.drop(['iupac'],axis=1)

# new_species

In [36]:
new_aqsol

,id,dataset,setnumber,name,inchi,inchikey,smiles,solubility,sd,ocurrences,species_id
0,1,A,3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.000000,1,4633
1,2,A,4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.000000,1,4634
2,3,A,5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.000000,1,4635
3,4,A,8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.000000,1,4636
4,5,A,9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.000000,1,4637
...,...,...,...,...,...,...,...,...,...,...,...
9977,9978,I,84,tetracaine,InChI=1S/C15H24N2O2/c1-4-5-10-16-14-8-6-13(7-9...,GKCBAIGFKIBETG-UHFFFAOYSA-N,C(c1ccc(cc1)NCCCC)(=O)OCCN(C)C,-3.010000,0.000000,1,12206
9978,9979,I,85,tetracycline,InChI=1S/C22H24N2O8/c1-21(31)8-5-4-6-11(25)12(...,OFVLGDICTFRJMM-WESIUVDSSA-N,OC1=C(C(C2=C(O)[C@@](C(C(C(N)=O)=C(O)[C@H]3N(C...,-2.930000,0.000000,1,12207
9979,9980,I,86,thymol,InChI=1S/C10H14O/c1-7(2)9-5-4-8(3)6-10(9)11/h4...,MGSRCZKZVOBKFT-UHFFFAOYSA-N,c1(cc(ccc1C(C)C)C)O,-2.190000,0.019222,3,825
9980,9981,I,93,verapamil,"InChI=1S/C27H38N2O4/c1-20(2)27(19-28,22-10-12-...",SGTNSNPWRIOYBX-UHFFFAOYSA-N,COc1ccc(CCN(C)CCCC(C#N)(C(C)C)c2ccc(OC)c(OC)c2...,-3.980000,0.000000,1,12208


In [42]:
# new_aqsol.to_csv('new_aqsol.csv',index=None)
# new_species.to_csv('new_species.csv',index=None)

In [32]:
# ids = []
# for c, key in enumerate(new_species['inchikey']):
#     id = int(species[species['inchikey']==key]['id'])
#     # print(id)
#     ids.append(id)
#     # matched_aqsol['species_id'][c] = id

# matched_aqsol['species_id'] = ids
# matched_aqsol.head()

In [33]:
print(unmatched_hlc.shape[0] + matched_aqsol.shape[0] + unmatched_aqsol.shape[0])
print(species.shape[0] + unmatched_aqsol.shape[0])

12208
12208


In [34]:
testboth = 'JZHGRUMIRATHIU-UHFFFAOYSA-N'
testhl = 'MYMOFIZGZYHOMD-UHFFFAOYSA-N	'
testaq = ''

In [35]:
# write a function that checks if species_id matches w inchikeys in both aqsol and hlc
def id_checker(db):
    pass